In [15]:
import json
import yaml
import os.path as op
import os
from yaml.loader import SafeLoader

In [7]:
def read_yaml(path_to_config_file):
    """
    Input:
    the path to the config file

    Returns
    a dictionary that contains all the config info

    """
    with open(path_to_config_file, "r") as v:
        config = yaml.load(v, Loader=SafeLoader)

    """     container = config["general"]["container"]
        host = config["general"]["host"]
        njobs = config["host_options"][host]["njobs"]
        if njobs == "" or njobs is None:
            njobs = 2
        host_str = f"{host}"
        if host == "local":
            launch_mode = config["host_options"]["local"]["launch_mode"]
            valid_options = ["serial", "parallel","dask_worker"]
            if launch_mode in valid_options:
                host_str = (
                    f"{host_str}, and commands will be launched in {launch_mode} mode "
                    f"every {njobs} jobs. "
                    f"Serial is safe but it will take longer. "
                    f"If you launch in parallel be aware that some of the "
                    f"processes might be killed if the limit (usually memory) "
                    f"of the machine is reached. "
                )
            else:
                die(
                    f"local:launch_mode {launch_mode} was passed, valid options are {valid_options}"
                )

        logger.warning(
            "\n"
            + "#####################################################\n"
            + f"Successfully read the config file {path_to_config_file} \n"
            + f'Basedir is: {config["general"]["basedir"]} \n'
            + f'Container is: {container}_{config["container_specific"][container]["version"]} \n'
            + f"Host is: {host_str} \n"
            + f'analysis folder is: {config["general"]["analysis_name"]} \n'
            + f"##################################################### \n"
        )
    """
    return config

In [8]:
lc_config=read_yaml('/Users/tiger/soft/launchcontainers/example_configs/0.3.0/example_config.yaml')
container="freesurferator"

In [31]:
yaml_info=lc_config["container_specific"][container]
fs_json_keys=['pre_fs','control_points','annotfile', 'mniroizip']
fs_json_val=['pre_fs/existingFS.zip','control_points/control.dat','mniroizip/mniroizip.zip','annotfile/annotfile.zip']

fs_json_dict= {key: value for key, value in zip(fs_json_keys, fs_json_val)}
json_template={'anat': 
                    {'location': {
                        'path': '/flywheel/v0/input/anat/T1.nii.gz', 
                        'name': 'T1.nii.gz',
                     },
                    'base': 'file'}
                    }

In [36]:
for key in fs_json_dict.keys():
    if key in yaml_info.keys() and yaml_info[key]:
        json_template[key] = {
                'location': {
                    'path': op.join('/flywheel/v0/input', fs_json_dict[key]), 
                    'name': op.basename(fs_json_dict[key])
                },
                'base': 'file'
            }
    if 'anat' in json_template.keys() and 'pre_fs' in json_template.keys():
        json_template.pop('anat')

In [37]:
json_template

{'pre_fs': {'location': {'path': '/flywheel/v0/input/pre_fs/existingFS.zip',
   'name': 'existingFS.zip'},
  'base': 'file'},
 'control_points': {'location': {'path': '/flywheel/v0/input/control_points/control.dat',
   'name': 'control.dat'},
  'base': 'file'}}

In [17]:
fs_json_dict

{'pre_fs_True': 'pre_fs/existingFS.zip',
 'pre_fs_False': 'anat/T1.nii.gz',
 'control_points': 'control_points/control.dat',
 'annotfile': 'mniroizip/mniroizip.zip',
 'mniroizip': 'annotfile/annotfile.zip'}

In [20]:
fs_json_dict.keys()

dict_keys(['pre_fs_True', 'pre_fs_False', 'control_points', 'annotfile', 'mniroizip'])

In [22]:
yaml_info.keys()

dict_keys(['version', 'pre_fs', 'control_points', 'source_path_fszip', 'precontainer_anat', 'anat_analysis_name', 'precontainer_fmriprep', 'fmriprep_analysis_name', 'prefs_zipname', 'prefs_unzipname', 'annotfile', 'mniroizip'])

In [40]:
preproc_json_keys=['ANAT','BVAL','BVEC', 'DIFF','FSMASK']
preproc_json_val=['ANAT/T1.nii.gz','BVAL/dwiF.bval','BVEC/dwiF.bvec','DIFF/dwiF.nii.gz','FSMASK/brainmask.nii.gz']


In [59]:
container="rtp2-pipeline"
pipeline_json_keys=['anatomical','bval','bvec', 'dwi','fs','tractparams']
pipeline_json_val=['anatomical/T1.nii.gz','bval/dwi.bval','bvec/dwi.bvec','dwi/dwi.nii.gz','fs/fs.zip','tractparams/tractparams.csv']


In [65]:
def get_config_dict(container,lc_config,rtp2_json_keys,rtp2_json_val):
    config_info_dict = {}
    yaml_info=lc_config["container_specific"][container]
    
    rtp2_json_dict= {key: value for key, value in zip(rtp2_json_keys, rtp2_json_val)}

    
    if container == "freesurferator":
        config_json_extra={'anat': 
                    {'location': {
                        'path': '/flywheel/v0/input/anat/T1.nii.gz', 
                        'name': 'T1.nii.gz',
                    },
                    'base': 'file'}
                    }
        for key in rtp2_json_dict.keys():
            if key in yaml_info.keys() and yaml_info[key]:
                config_json_extra[key] = {
                        'location': {
                            'path': op.join('/flywheel/v0/input', rtp2_json_dict[key]), 
                            'name': op.basename(rtp2_json_dict[key])
                        },
                        'base': 'file'
                    }
            if 'anat' in config_json_extra.keys() and 'pre_fs' in config_json_extra.keys():
                config_json_extra.pop('anat')

    else:
        config_json_extra={}
        for key in rtp2_json_dict.keys():
            config_json_extra[key] = {
                    'location': {
                        'path': op.join('/flywheel/v0/input', rtp2_json_dict[key]), 
                        'name': op.basename(rtp2_json_dict[key])
                    },
                    'base': 'file'
                }
        
    config_info_dict['input']= config_json_extra   
    return config_info_dict

In [66]:
config_info_dict=get_config_dict(container,lc_config,pipeline_json_keys,pipeline_json_val)

In [67]:
config_info_dict

{'input': {'anatomical': {'location': {'path': '/flywheel/v0/input/anatomical/T1.nii.gz',
    'name': 'T1.nii.gz'},
   'base': 'file'},
  'bval': {'location': {'path': '/flywheel/v0/input/bval/dwi.bval',
    'name': 'dwi.bval'},
   'base': 'file'},
  'bvec': {'location': {'path': '/flywheel/v0/input/bvec/dwi.bvec',
    'name': 'dwi.bvec'},
   'base': 'file'},
  'dwi': {'location': {'path': '/flywheel/v0/input/dwi/dwi.nii.gz',
    'name': 'dwi.nii.gz'},
   'base': 'file'},
  'fs': {'location': {'path': '/flywheel/v0/input/fs/fs.zip',
    'name': 'fs.zip'},
   'base': 'file'},
  'tractparams': {'location': {'path': '/flywheel/v0/input/tractparams/tractparams.csv',
    'name': 'tractparams.csv'},
   'base': 'file'}}}

In [71]:
config_info_dict['cc']

KeyError: 'cc'

In [74]:
if "input" in config_info_dict:
    print("in")

in


In [2]:
import os.path as op
mni_roi="/home/tlei/tlei/FG.nii.gz"
file_name = op.basename(mni_roi)
file_ext = op.splitext(file_name)[1]

In [3]:
file_name

'FG.nii.gz'

In [4]:
file_ext

'.gz'

In [5]:
op.splitext(file_name)

('FG.nii', '.gz')

In [10]:
lc_config_path='/media/tlei/data/soft/launchcontainers/example_configs/0.3.tlei/new_lc_config.yaml'
lc_config=read_yaml(lc_config_path)

In [11]:
lc_config

{'general': {'basedir': '/home/tlei/tlei/LMC_DWI_course',
  'bidsdir_name': 'BIDS',
  'containerdir': '/home/tlei/tlei/LMC_DWI_course/containers',
  'container': 'freesurferator',
  'analysis_name': 'test_on_msi01_for_preparedwi',
  'host': 'local',
  'force': True,
  'print_command_only': False,
  'log_dir': 'analysis_dir',
  'log_filename': 'lc_log'},
 'container_specific': {'anatrois': {'version': '4.6.1-7.3.2',
   'pre_fs': True,
   'source_fszip': 'anatrois',
   'prefs_dir_name': 'anatrois_4.6.1-7.3.2',
   'prefs_analysis_name': '6prefs_from_fmriprep',
   'prefs_zipname': '^anatrois_S.*\\.zip$',
   'annotfile': None,
   'mniroizip': None},
  'rtppreproc': {'version': '1.2.0-3.0.3',
   'rpe': True,
   'precontainer_anat': 'anatrois_4.6.1-7.3.2',
   'anat_analysis_name': 'fMRIprep_brainmask'},
  'rtp-pipeline': {'version': '4.5.2-3.0.3',
   'precontainer_anat': 'anatrois_4.6.1-7.3.2',
   'anat_analysis_name': 'fulltract_anatrerun',
   'precontainer_preproc': 'rtppreproc_1.2.0-3.0.3'

In [13]:
    # read parameters from lc_config
    basedir = lc_config['general']['basedir']
    container = lc_config['general']['container']
    force = lc_config["general"]["force"]
    analysis_name= lc_config['general']['analysis_name']
    run_lc = False
    force= force or run_lc    
    version = lc_config["container_specific"][container]["version"]    
    bidsdir_name = lc_config['general']['bidsdir_name']  
    container_folder = op.join(basedir, bidsdir_name,'derivatives',f'{container}_{version}')
    if not op.isdir(container_folder):
        os.makedirs(container_folder)

In [16]:
    analysis_dir = op.join(
        container_folder,
        f"analysis-{analysis_name}",
                )
    if not op.isdir(analysis_dir):
        os.makedirs(analysis_dir)

In [17]:
  container_configs_under_analysis_folder = op.join(analysis_dir,os.path.basename('/media/tlei/data/soft/launchcontainers/example_configs/container_specific_example_configs/freesurferator/0.2.0_7.4.1rc19/example_config.json'))

In [18]:
container_configs_under_analysis_folder

'/home/tlei/tlei/LMC_DWI_course/BIDS/derivatives/freesurferator_0.2.0-7.4.1rc19/analysis-test_on_msi01_for_preparedwi/example_config.json'

In [33]:
    container_specific_configs={}
    container_specific_configs['config']=container_configs_under_analysis_folder
    

In [39]:
container_specific_configs

{'config': '/home/tlei/tlei/LMC_DWI_course/BIDS/derivatives/freesurferator_0.2.0-7.4.1rc19/analysis-test_on_msi01_for_preparedwi/example_config.json',
 'freesurferator': {'ANAT': 'ANAT/T1.nii.gz',
  'BVAL': 'BVAL/dwiF.bval',
  'BVEC': 'BVEC/dwiF.bvec',
  'DIFF': 'DIFF/dwiF.nii.gz',
  'FSMASK': 'FSMASK/brain.nii.gz'}}

In [41]:
container_specific_configs[container]['anat']['location']['name']

dict_keys(['ANAT', 'BVAL', 'BVEC', 'DIFF', 'FSMASK'])

In [36]:
        preproc_json_keys=['ANAT','BVAL','BVEC', 'DIFF','FSMASK']
        preproc_json_val=['ANAT/T1.nii.gz','BVAL/dwiF.bval','BVEC/dwiF.bvec','DIFF/dwiF.nii.gz','FSMASK/brain.nii.gz']
        
        container_specific_configs[container]= {key: value for key, value in zip(preproc_json_keys, preproc_json_val)}
        

In [38]:
container_specific_configs[container]

{'ANAT': 'ANAT/T1.nii.gz',
 'BVAL': 'BVAL/dwiF.bval',
 'BVEC': 'BVEC/dwiF.bvec',
 'DIFF': 'DIFF/dwiF.nii.gz',
 'FSMASK': 'FSMASK/brain.nii.gz'}

In [31]:
    def process_optional_input(container,file_path, analysis_dir, option=None):
        if os.path.isfile(file_path):
            logger.info("\n"
                +f" You have choossen to pass  {file_path} to {container}, it will be first copy to {analysis_dir}")
        else:
            logger.error("\n"
                        f"{file_path} does not exist")        
        
        file_name = os.path.basename(file_path)
        file_ext = os.path.splitext(file_name)[1]
        
        if container in ['anatrois','freesurferator']:
            if file_ext in ['.nii', '.gz','.zip']:
                do.copy_file(file_path, os.path.join(analysis_dir, file_name), force)          
            else:
                raise ValueError("Unsupported file type.")
        if container in ['rtp2-preproc']:
            if file_ext in ['.nii', '.gz']:
                do.copy_file(file_path, os.path.join(analysis_dir, file_name), force)                          
            else:
                raise ValueError("Unsupported file type.")
                    
        if container in ['rtp2-pipeline']:
            if option == "tractparams":
                if file_ext in ['.csv']:    
                    do.copy_file(file_path, os.path.join(analysis_dir, file_name), force)
                else:
                    raise ValueError("Unsupported file type.")                    
            if option == "fsmask":
                if file_ext in ['.nii', '.gz']:
                    do.copy_file(file_path, os.path.join(analysis_dir, file_name), force)            
                else:
                    raise ValueError("Unsupported file type.")                
            if option == "qmap_zip":
                if file_ext == '.zip':
                    do.copy_file(file_path, os.path.join(analysis_dir, file_name), force)          
                else:
                    raise ValueError("Unsupported file type.")
        return file_name
    
    # copy annotfile or mnizip file to analysis folder
    if container in ['anatrois']:
        pre_fs= lc_config["container_specific"][container]["pre_fs"]
        annotfile = lc_config["container_specific"][container]["annotfile"]
        mniroizip = lc_config["container_specific"][container]["mniroizip"]
        if pre_fs:
            file_name="existingFS.zip"
            container_specific_configs[container]['pre_fs']=f"pre_fs/{file_name}"         
        if annotfile:
            file_name=process_optional_input(container,annotfile,analysis_dir)
            container_specific_configs[container]['annotfile']=f"annotfile/{file_name}"
        if mniroizip:
            file_name=process_optional_input(container,mniroizip,analysis_dir)
            container_specific_configs[container]['mniroizip']=f"mniroizip/{file_name}"
        # copy annotfile or mnizip file to analysis folder
    if container in ['freesurferator']:
        pre_fs= lc_config["container_specific"][container]["pre_fs"]
        control_points= lc_config["container_specific"][container]["control_points"]
        annotfile = lc_config["container_specific"][container]["annotfile"]
        mniroizip = lc_config["container_specific"][container]["mniroizip"]
        if pre_fs:
            file_name="existingFS.zip"
            container_specific_configs[container]['pre_fs']=f"pre_fs/{file_name}"
        if control_points:
            file_name="control.dat"
            container_specific_configs[container]['control_points']=f"control_points/{file_name}"            
        if annotfile:
            file_name=process_optional_input(container,annotfile,analysis_dir)
            container_specific_configs[container]['annotfile']=f"annotfile/{file_name}"
        if mniroizip:
            file_name=process_optional_input(container,mniroizip,analysis_dir)
            container_specific_configs[container]['mniroizip']=f"mniroizip/{file_name}"
    # copy qmap.nii of qmap.nii.gz to analysis folder
    if container in ['rtppreproc']:
        preproc_json_keys=['ANAT','BVAL','BVEC', 'DIFF','FSMASK']
        preproc_json_val=['ANAT/T1.nii.gz','BVAL/dwiF.bval','BVEC/dwiF.bvec','DIFF/dwiF.nii.gz','FSMASK/brain.nii.gz']
        container_specific_configs[container]= {key: value for key, value in zip(preproc_json_keys, preproc_json_val)}
        rpe=lc_config["container_specific"][container]["rpe"]
        if rpe:
            container_specific_configs[container]['RBVC']= 'RBVC/dwiR.bvec'
            container_specific_configs[container]['RBVL']= 'RBVL/dwiR.bval'
            container_specific_configs[container]['RDIF']= 'RDIF/dwiR.nii.gz'            
    if container in ['rtp2-preproc']:
        preproc_json_keys=['ANAT','BVAL','BVEC', 'DIFF','FSMASK']
        preproc_json_val=['ANAT/T1.nii.gz','BVAL/dwiF.bval','BVEC/dwiF.bvec','DIFF/dwiF.nii.gz','FSMASK/brain.nii.gz']
        container_specific_configs[container]= {key: value for key, value in zip(preproc_json_keys, preproc_json_val)}
        
        rpe=lc_config["container_specific"][container]["rpe"]
        qmap=lc_config["container_specific"][container]["qmap_nifti"]
        if rpe:
            container_specific_configs[container]['RBVC']= 'RBVC/dwiR.bvec'
            container_specific_configs[container]['RBVL']= 'RBVL/dwiR.bval'
            container_specific_configs[container]['RDIF']= 'RDIF/dwiR.nii.gz'            
        if qmap:
            file_name=process_optional_input(container,qmap,analysis_dir)
            container_specific_configs[container]['qmap']=f"qmap/{file_name}"
        
    if container in ['rtp-pipeline']:
        pipeline_json_keys=['anatomical','bval','bvec', 'dwi','fs']
        pipeline_json_val=['anatomical/T1.nii.gz','bval/dwi.bval','bvec/dwi.bvec','dwi/dwi.nii.gz','fs/fs.zip']
        container_specific_configs[container]= {key: value for key, value in zip(pipeline_json_keys, pipeline_json_val)}
        
        tractparams=lc_config["container_specific"][container]["tractparams"]
        if tractparams:
            file_name=process_optional_input(container,tractparams,analysis_dir,"tractparams")    
            container_specific_configs[container]['tractparams']=f"tractparams/{file_name}"
    if container in ['rtp2-pipeline']:
        pipeline_json_keys=['anatomical','bval','bvec', 'dwi','fs']
        pipeline_json_val=['anatomical/T1.nii.gz','bval/dwi.bval','bvec/dwi.bvec','dwi/dwi.nii.gz','fs/fs.zip']
        container_specific_configs[container]= {key: value for key, value in zip(pipeline_json_keys, pipeline_json_val)}        
        tractparams=lc_config["container_specific"][container]["tractparams"]
        fsmask=lc_config["container_specific"][container]["fsmask"]
        qmap_zip=lc_config["container_specific"][container]["qmap_zip"]
        if tractparams:
            file_name=process_optional_input(container,tractparams,analysis_dir,"tractparams")
            container_specific_configs[container]['tractparams']=f"tractparams/{file_name}"
        if fsmask:
            file_name=process_optional_input(container,fsmask,analysis_dir,"fsmask")
            container_specific_configs[container]['fsmask']=f"fsmask/{file_name}"
        if qmap_zip:
            file_name=process_optional_input(container,qmap_zip,analysis_dir,"qmap_zip")           
            container_specific_configs[container]['qmap_zip']=f"qmap_zip/{file_name}"       

In [23]:
container

'freesurferator'

In [42]:
dict= {
    "inputs": {
        "anat": {
            "location": {
                "path": "/flywheel/v0/input/anat/T1.nii.gz",
                "name": "T1.nii.gz"
            },
            "base": "file"
        },
        "pre_fs": {
            "location": {
                "path": "/flywheel/v0/input/pre_fs/existingFS.zip",
                "name": "existingFS.zip"
            },
            "base": "file"
        },
        "control_points": {
            "location": {
                "path": "/flywheel/v0/input/control_points/control.dat",
                "name": "control.dat"
            },
            "base": "file"
        },
        "mniroizip": {
            "location": {
                "path": "/flywheel/v0/input/mniroizip/FG.nii.gz",
                "name": "FG.nii.gz"
            },
            "base": "file"
        }
}
}

In [44]:
dict['inputs']['anat']['location']['name']

'T1.nii.gz'

In [50]:

from bids import BIDSLayout
layout=BIDSLayout(os.path.join('/home/tlei/tlei/multishell_dwi/nifti'))


In [57]:
dwi_dir=layout.get(subject= "SM003", session="001", extension='nii.gz',suffix= 'dwi', direction="AP", return_type='filename')

In [58]:
dwi_dir

['/home/tlei/tlei/multishell_dwi/nifti/sub-SM003/ses-001/dwi/sub-SM003_ses-001_acq-b1000_dir-AP_dwi.nii.gz',
 '/home/tlei/tlei/multishell_dwi/nifti/sub-SM003/ses-001/dwi/sub-SM003_ses-001_acq-b2500_dir-AP_dwi.nii.gz']

In [ ]:
if len(dwi_dir) > 1:
    dwi_acq = [f for f in dwi_dir if 'acq-' in f]
    if len(dwi_acq) == 0:
        print(f"No files with different acq- to concatenate.\n")
    elif len(dwi_acq) == 1:
        print(f"Found only {dwi_acq[0]} to concatenate. There must be at least two files with different acq.\n")
    elif len(dwi_acq) > 1:
        if not os.path.isfile(srcFileDwi_nii):
            print(f"Concatenating with mrcat of mrtrix3 these files: {dwi_acq} in: {srcFileDwi_nii} \n")
            dwi_acq.sort()
            sp.run(['mrcat',*dwi_acq,srcFileDwi_nii])

In [59]:
*dwi_acq

SyntaxError: can't use starred expression here (162654131.py, line 1)